In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch.serialization
from omegaconf import dictconfig, base
import typing
import collections

In [2]:
!pip install pytorch-tabular


In [3]:
data=pd.read_csv("/content/data_final preprocessed_cdms.csv")
data

,cat__Varicella_1,cat__Varicella_2,cat__Mono or polysymptomatic_2,cat__Oligoclonal bands_0,cat__Oligoclonal bands_1,cat__Oligoclonal bands_2,cat__initial EDSS_1.0,cat__initial EDSS_2.0,cat__initial symptom_1.0,cat__initial symptom_2.0,...,cat__initial symptom_nan,cat__ final EDSS_1.0,cat__ final EDSS_2.0,remainder__Gender,remainder__LLSSEP,remainder__VEP,remainder__BAEP,remainder__Periventricular MRI,remainder__Infratentorial MRI,remainder__group
0,1,0,0,1,0,0,1,0,0,1,...,0,1,0,1,1,0,0,0,0,1
1,0,1,1,0,1,0,0,1,0,0,...,0,0,1,1,1,1,0,0,0,1
2,1,0,0,0,1,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
3,1,0,1,0,1,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,1
4,1,0,1,1,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,0,0,1,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
268,0,0,1,0,1,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,0
269,0,0,1,1,0,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0
270,0,0,1,0,1,0,1,0,0,0,...,0,1,0,0,1,1,0,1,1,0


In [4]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import NodeConfig
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig, ExperimentConfig
from sklearn.model_selection import train_test_split


In [5]:
X = data.drop(columns=["remainder__group"])
y = data["remainder__group"]

In [6]:
# =====================================================
# 4. Train / Test split (EXPLICIT)
# =====================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
)

In [7]:
# !pip install --upgrade imbalanced-learn
from imblearn.over_sampling import BorderlineSMOTE
Borderline = BorderlineSMOTE(random_state=42)

X_train_res, y_train_res = Borderline.fit_resample(X_train, y_train)


In [8]:


train_df = pd.concat([X_train_res, y_train_res], axis=1)
test_df  = pd.concat([X_test, y_test], axis=1)

In [9]:
data_config = DataConfig(target=["remainder__group"], continuous_cols=data.drop(columns=["remainder__group"]).columns.tolist())

In [10]:
model_config = NodeConfig(
    task="classification",
    num_trees=512,
    depth=6,
    num_layers=1, learning_rate=1e-3
)


In [11]:
trainer_config = TrainerConfig(
    max_epochs=50,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    # project_name="node_experiment",
    checkpoints=None        # ✅ FIXES PyTorch 2.6 unpickling error
)


# ===============================
# 7. Optimizer configuration
# ===============================
optimizer_config = OptimizerConfig()
model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    trainer_config=trainer_config,
    optimizer_config=optimizer_config
)


INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [12]:
model.fit(train=train_df)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
/usr/local/lib/python3.12/dist-packages/pytorch_tabular/tabular_datamodule.py:386: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.19068112 -0.83985543  1.19068112 -0.83985543 -0.83985543  1.19068112
 -0.83985543 -0.83985543 -0.83985543  1.19068112 -0.83985543  1.19068112
 -0.83985543 -0.83985543  1.19068112  1.19068112  1.19068112 -0.83985543
  1.19068112  1.19068112  1.19068112 -0.83985543  1.19068112 -0.83985543
  1.19068112  1.19068112 -0.83985543  1.19068112 -0.83985543 -0.83985543
 -0.83985543  1.19068112 -0.83985543 -0.83985543  1.19068112 -0.83985543
 -0.83985543 -0.83985543 -0.83985543 -0.83985543 -0.83985543  1.19068112
  1.19068112  1.19068112 -0.83985543 -0.83985543  1.19068112  1.19068112
  1.19068112

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ NODEBackbone     │  253 K │ train │
│ 1 │ _embedding_layer │ Embedding1dLayer │     54 │ train │
│ 2 │ _head            │ Lambda           │      0 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 252 K                                                                                            
Non-trainable params: 769                                                                                          
Total params: 253 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 9                                                                                           
Modules in eval mode: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (3)
is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you 
want to see logs for the training epoch.

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


INFO:pytorch_tabular.tabular_model:Training the model completed


In [13]:
model.model.eval()

X_test_tensor = torch.tensor(
    X_test.values,
    dtype=torch.float32
)

if torch.cuda.is_available():
    X_test_tensor = X_test_tensor.cuda()
    model.model.cuda()

with torch.no_grad():
    # Wrap X_test_tensor in a dictionary for the model's forward pass
    output = model.model({"continuous": X_test_tensor})
    logits = output['logits']

print("Logits shape:", logits.shape)
print("First 5 logits:\n", logits[:5])

Logits shape: torch.Size([55, 2])
First 5 logits:
 tensor([[-0.0462,  0.0271],
        [-0.1412,  0.1211],
        [-0.1258,  0.0997],
        [-0.0144, -0.0038],
        [ 0.0143, -0.0271]])


In [14]:
probs = torch.softmax(logits, dim=1)
preds = probs.argmax(dim=1)

print("First 5 probabilities:\n", probs[:5])
print("First 5 predictions:\n", preds[:5])

First 5 probabilities:
 tensor([[0.4817, 0.5183],
        [0.4348, 0.5652],
        [0.4439, 0.5561],
        [0.4974, 0.5026],
        [0.5103, 0.4897]])
First 5 predictions:
 tensor([1, 1, 1, 1, 0])


In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Accuracy:",
    accuracy_score(
        y_test.values,
        preds.cpu().numpy()
))


print(classification_report(
    y_test.values,
    preds.cpu().numpy()
))

print("Confusion Matrix:\n",
      confusion_matrix(
          y_test.values,
          preds.cpu().numpy()
      ))


Accuracy: 0.8363636363636363
              precision    recall  f1-score   support

           0       0.95      0.71      0.82        28
           1       0.76      0.96      0.85        27

    accuracy                           0.84        55
   macro avg       0.86      0.84      0.83        55
weighted avg       0.86      0.84      0.83        55

Confusion Matrix:
 [[20  8]
 [ 1 26]]


In [16]:
# model_config = NodeConfig(
#     task="classification",
#     num_trees=580,
#     depth=6,
#     num_layers=1, learning_rate=1e-3
# )



# trainer_config = TrainerConfig(
#     max_epochs=20,
#     accelerator="gpu" if torch.cuda.is_available() else "cpu",
#     # project_name="node_experiment",
#     checkpoints=None        # ✅ FIXES PyTorch 2.6 unpickling error
# )


# # ===============================
# # 7. Optimizer configuration
# # ===============================
# optimizer_config = OptimizerConfig()
# model = TabularModel(
#     data_config=data_config,
#     model_config=model_config,
#     trainer_config=trainer_config,
#     optimizer_config=optimizer_config
# )
# model.fit(train=train_df)

# model.model.eval()

# X_test_tensor = torch.tensor(
#     X_test.values,
#     dtype=torch.float32
# )

# if torch.cuda.is_available():
#     X_test_tensor = X_test_tensor.cuda()
#     model.model.cuda()

# with torch.no_grad():
#     # Wrap X_test_tensor in a dictionary for the model's forward pass
#     output = model.model({"continuous": X_test_tensor})
#     logits = output['logits']

# print("Logits shape:", logits.shape)
# print("First 5 logits:\n", logits[:5])

# probs = torch.softmax(logits, dim=1)
# preds = probs.argmax(dim=1)

# print("First 5 probabilities:\n", probs[:5])
# print("First 5 predictions:\n", preds[:5])
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# print("Accuracy:",
#     accuracy_score(
#         y_test.values,
#         preds.cpu().numpy()
# ))


# print(classification_report(
#     y_test.values,
#     preds.cpu().numpy()
# ))

# print("Confusion Matrix:\n",
#       confusion_matrix(
#           y_test.values,
#           preds.cpu().numpy()
#       ))
